# Phase 6: Deployment

**Objectives:**
1. Create inference function
2. Test prediction pipeline
3. Build Streamlit app
4. Create production folder structure
5. Generate requirements.txt
6. Deployment instructions

## 6.1 Import Libraries & Load Artifacts

In [ ]:
import numpy as np
import pandas as pd
import joblib
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load saved artifacts
model = joblib.load('best_model.pkl')
preprocessor = joblib.load('preprocessor.pkl')
feature_names = joblib.load('feature_names.pkl')

print("Loaded artifacts:")
print(f"  - Model: {type(model).__name__}")
print(f"  - Preprocessor: {type(preprocessor).__name__}")
print(f"  - Features: {len(feature_names)}")

## 6.2 Define Inference Function

In [ ]:
def predict_flight_price(airline, source, destination, additional_info,
                         duration_minutes, stops, journey_day, journey_month,
                         journey_day_of_week, dep_hour, dep_minute):
    """
    Predict flight price based on input features.
    
    Parameters:
    -----------
    airline : str
        Name of the airline
    source : str
        Departure city
    destination : str
        Arrival city
    additional_info : str
        Additional flight information
    duration_minutes : int
        Total flight duration in minutes
    stops : int
        Number of stops (0-4)
    journey_day : int
        Day of the month (1-31)
    journey_month : int
        Month (1-12)
    journey_day_of_week : int
        Day of week (0=Monday, 6=Sunday)
    dep_hour : int
        Departure hour (0-23)
    dep_minute : int
        Departure minute (0-59)
    
    Returns:
    --------
    float : Predicted price in INR
    """
    
    # Create input DataFrame
    input_data = pd.DataFrame({
        'Airline': [airline],
        'Source': [source],
        'Destination': [destination],
        'Additional_Info': [additional_info],
        'Duration_Minutes': [duration_minutes],
        'Stops': [stops],
        'Journey_Day': [journey_day],
        'Journey_Month': [journey_month],
        'Journey_Day_of_Week': [journey_day_of_week],
        'Dep_Hour': [dep_hour],
        'Dep_Minute': [dep_minute]
    })
    
    # Preprocess
    input_processed = preprocessor.transform(input_data)
    
    # Predict
    prediction = model.predict(input_processed)[0]
    
    return max(0, prediction)  # Ensure non-negative price

## 6.3 Test Inference Function

In [ ]:
# Test case 1: Budget flight
price1 = predict_flight_price(
    airline='IndiGo',
    source='Delhi',
    destination='Cochin',
    additional_info='No info',
    duration_minutes=180,  # 3 hours
    stops=0,
    journey_day=15,
    journey_month=5,
    journey_day_of_week=2,  # Wednesday
    dep_hour=10,
    dep_minute=30
)
print(f"Test 1 - Budget Non-stop IndiGo: ₹{price1:,.0f}")

In [ ]:
# Test case 2: Premium flight
price2 = predict_flight_price(
    airline='Jet Airways',
    source='Banglore',
    destination='New Delhi',
    additional_info='No info',
    duration_minutes=420,  # 7 hours
    stops=1,
    journey_day=20,
    journey_month=3,
    journey_day_of_week=6,  # Sunday
    dep_hour=8,
    dep_minute=0
)
print(f"Test 2 - Premium 1-stop Jet Airways: ₹{price2:,.0f}")

In [ ]:
# Test case 3: Red-eye budget flight
price3 = predict_flight_price(
    airline='SpiceJet',
    source='Kolkata',
    destination='Banglore',
    additional_info='No info',
    duration_minutes=150,  # 2.5 hours
    stops=0,
    journey_day=5,
    journey_month=4,
    journey_day_of_week=4,  # Friday
    dep_hour=2,  # Red-eye
    dep_minute=30
)
print(f"Test 3 - Red-eye Non-stop SpiceJet: ₹{price3:,.0f}")

In [ ]:
print("\nPrice Comparison:")
print(f"  Budget IndiGo (non-stop, morning): ₹{price1:,.0f}")
print(f"  Premium Jet Airways (1-stop, Sunday): ₹{price2:,.0f}")
print(f"  Budget SpiceJet (non-stop, red-eye): ₹{price3:,.0f}")
print(f"\nPrice range: ₹{min(price1, price2, price3):,.0f} - ₹{max(price1, price2, price3):,.0f}")

## 6.4 Create Streamlit App

In [ ]:
streamlit_app_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, time

# Page config
st.set_page_config(
    page_title="Flight Price Predictor",
    page_icon="✈️",
    layout="centered"
)

# Load model and preprocessor
@st.cache_resource
def load_artifacts():
    model = joblib.load('best_model.pkl')
    preprocessor = joblib.load('preprocessor.pkl')
    return model, preprocessor

model, preprocessor = load_artifacts()

# Title
st.title("✈️ Flight Price Predictor")
st.markdown("Predict flight prices in India based on route, airline, and timing.")
st.markdown("---")

# Sidebar info
st.sidebar.header("About")
st.sidebar.info(
    "This app predicts flight prices using a machine learning model "
    "trained on historical flight data.\\n\\n"
    "**Model:** XGBoost\\n"
    "**Accuracy:** 91.7% R²\\n"
    "**Avg Error:** ±₹659"
)

# Input columns
col1, col2 = st.columns(2)

with col1:
    st.subheader("Flight Details")
    
    airline = st.selectbox(
        "Airline",
        options=['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet', 'Vistara', 
                 'GoAir', 'Air Asia', 'Multiple carriers', 
                 'Multiple carriers Premium economy', 'Vistara Premium economy',
                 'Trujet']
    )
    
    source = st.selectbox(
        "Source (Departure City)",
        options=['Delhi', 'Kolkata', 'Banglore', 'Mumbai', 'Chennai']
    )
    
    destination = st.selectbox(
        "Destination (Arrival City)",
        options=['Cochin', 'Banglore', 'Delhi', 'New Delhi', 'Hyderabad', 'Kolkata']
    )
    
    stops = st.selectbox(
        "Number of Stops",
        options=[0, 1, 2, 3, 4],
        format_func=lambda x: 'Non-stop' if x == 0 else f'{x} stop{"s" if x > 1 else ""}'
    )

with col2:
    st.subheader("Journey Details")
    
    journey_date = st.date_input(
        "Journey Date",
        value=datetime.now()
    )
    
    dep_time = st.time_input(
        "Departure Time",
        value=time(10, 0)
    )
    
    # Duration input
    duration_hours = st.number_input("Duration (Hours)", min_value=0, max_value=48, value=2)
    duration_minutes_input = st.number_input("Duration (Minutes)", min_value=0, max_value=59, value=30)
    
    additional_info = st.selectbox(
        "Additional Info",
        options=['No info', 'In-flight meal not included', 'No check-in baggage included',
                 '1 Long layover', 'Change airports', 'Business class']
    )

# Calculate derived features
duration_minutes = duration_hours * 60 + duration_minutes_input
journey_day = journey_date.day
journey_month = journey_date.month
journey_day_of_week = journey_date.weekday()
dep_hour = dep_time.hour
dep_minute = dep_time.minute

st.markdown("---")

# Predict button
if st.button("🔮 Predict Price", type="primary", use_container_width=True):
    
    # Validate source != destination
    if source == destination or (source == 'Delhi' and destination == 'New Delhi'):
        st.error("Source and Destination cannot be the same!")
    else:
        # Create input DataFrame
        input_data = pd.DataFrame({
            'Airline': [airline],
            'Source': [source],
            'Destination': [destination],
            'Additional_Info': [additional_info],
            'Duration_Minutes': [duration_minutes],
            'Stops': [stops],
            'Journey_Day': [journey_day],
            'Journey_Month': [journey_month],
            'Journey_Day_of_Week': [journey_day_of_week],
            'Dep_Hour': [dep_hour],
            'Dep_Minute': [dep_minute]
        })
        
        # Preprocess and predict
        try:
            input_processed = preprocessor.transform(input_data)
            prediction = model.predict(input_processed)[0]
            prediction = max(0, prediction)  # Ensure non-negative
            
            # Display result
            st.success("Prediction Complete!")
            
            col1, col2, col3 = st.columns(3)
            col1.metric("Predicted Price", f"₹{prediction:,.0f}")
            col2.metric("Price Range (±MAE)", f"₹{max(0,prediction-659):,.0f} - ₹{prediction+659:,.0f}")
            col3.metric("Confidence", "91.7%")
            
            # Flight summary
            st.markdown("### Flight Summary")
            st.markdown(f"""
            | Detail | Value |
            |--------|-------|
            | Route | {source} → {destination} |
            | Airline | {airline} |
            | Stops | {'Non-stop' if stops == 0 else f'{stops} stop(s)'} |
            | Duration | {duration_hours}h {duration_minutes_input}m |
            | Date | {journey_date.strftime('%d %b %Y')} ({['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][journey_day_of_week]}) |
            | Departure | {dep_time.strftime('%H:%M')} |
            """)
            
        except Exception as e:
            st.error(f"Prediction failed: {str(e)}")

# Tips section
st.markdown("---")
with st.expander("💡 Tips to Get Cheaper Flights"):
    st.markdown("""
    Based on our analysis:
    
    1. **Book non-stop flights** - They're often cheaper than connecting flights
    2. **Choose budget airlines** - SpiceJet, GoAir, IndiGo, Air Asia
    3. **Fly red-eye (0-4 AM)** - Cheapest departure times
    4. **Avoid Sundays** - Most expensive day to fly
    5. **Book in April** - Cheapest month in our data
    6. **Chennai-Kolkata** - One of the cheapest routes
    """)

# Footer
st.markdown("---")
st.markdown(
    "<div style='text-align: center; color: gray;'>"
    "Built with Streamlit | Model: XGBoost | Data: Flight prices in India"
    "</div>",
    unsafe_allow_html=True
)
'''

# Save Streamlit app
with open('app.py', 'w') as f:
    f.write(streamlit_app_code)

print("Streamlit app saved to 'app.py'")

## 6.5 Create Requirements File

In [ ]:
requirements = '''streamlit>=1.28.0
pandas>=2.0.0
numpy>=1.24.0
scikit-learn>=1.3.0
xgboost>=2.0.0
joblib>=1.3.0
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("requirements.txt created")
print(requirements)

## 6.6 Create Production Folder Structure

In [ ]:
# Create deployment folder
deploy_folder = 'flight_price_predictor'
os.makedirs(deploy_folder, exist_ok=True)

print(f"Created folder: {deploy_folder}/")

In [ ]:
# Copy necessary files to deployment folder
import shutil

files_to_copy = [
    'app.py',
    'best_model.pkl',
    'preprocessor.pkl',
    'requirements.txt'
]

for file in files_to_copy:
    if os.path.exists(file):
        shutil.copy(file, os.path.join(deploy_folder, file))
        print(f"Copied: {file}")
    else:
        print(f"Warning: {file} not found")

In [ ]:
# Create README for deployment
readme_content = '''# Flight Price Predictor

A machine learning application to predict flight prices in India.

## Model Performance

- **Model**: XGBoost (Tuned)
- **Test R²**: 0.9167 (91.7% variance explained)
- **Test RMSE**: ₹1,163
- **Test MAE**: ₹659
- **Test MAPE**: 7.95%

## Features Used

- Airline
- Source (Departure City)
- Destination (Arrival City)
- Duration (Minutes)
- Number of Stops
- Journey Day, Month, Day of Week
- Departure Hour, Minute
- Additional Info

## Local Setup

1. Install dependencies:
```bash
pip install -r requirements.txt
```

2. Run the app:
```bash
streamlit run app.py
```

3. Open browser at `http://localhost:8501`

## Deploy to Streamlit Cloud

1. Push this folder to GitHub
2. Go to [share.streamlit.io](https://share.streamlit.io)
3. Connect your GitHub repository
4. Select `app.py` as the main file
5. Deploy!

## Files

- `app.py` - Streamlit application
- `best_model.pkl` - Trained XGBoost model
- `preprocessor.pkl` - Sklearn preprocessor (scaler + encoder)
- `requirements.txt` - Python dependencies

## Key Insights

1. **Non-stop flights** are often cheaper than connecting flights
2. **Budget airlines** (SpiceJet, GoAir, IndiGo) offer lower prices
3. **Red-eye flights** (0-4 AM) are the cheapest
4. **Sundays** are the most expensive day to fly
5. **April** is the cheapest month
6. **Jet Airways** commands the highest premium
'''

with open(os.path.join(deploy_folder, 'README.md'), 'w') as f:
    f.write(readme_content)

print("README.md created")

In [ ]:
# List deployment folder contents
print(f"\nDeployment folder structure:")
print(f"{deploy_folder}/")
for file in os.listdir(deploy_folder):
    file_path = os.path.join(deploy_folder, file)
    size = os.path.getsize(file_path) / 1024  # KB
    print(f"  ├── {file} ({size:.1f} KB)")

## 6.7 Test App Locally (Instructions)

In [ ]:
print("="*60)
print("HOW TO RUN THE APP LOCALLY")
print("="*60)
print("""
1. Open terminal/command prompt

2. Navigate to the deployment folder:
   cd flight_price_predictor

3. Install dependencies:
   pip install -r requirements.txt

4. Run the Streamlit app:
   streamlit run app.py

5. Open your browser at:
   http://localhost:8501
""")

## 6.8 Streamlit Cloud Deployment (Instructions)

In [ ]:
print("="*60)
print("HOW TO DEPLOY TO STREAMLIT CLOUD")
print("="*60)
print("""
1. Create a GitHub repository

2. Upload these files to the repository:
   - app.py
   - best_model.pkl
   - preprocessor.pkl
   - requirements.txt

3. Go to: https://share.streamlit.io

4. Sign in with GitHub

5. Click "New app"

6. Select:
   - Repository: your-username/your-repo-name
   - Branch: main
   - Main file path: app.py

7. Click "Deploy!"

8. Wait for deployment (2-5 minutes)

9. Your app will be live at:
   https://your-app-name.streamlit.app
""")

## 6.9 Phase 6 Summary (Upload This Output)

In [ ]:
print("="*70)
print("PHASE 6: DEPLOYMENT SUMMARY")
print("="*70)

print("\n1. INFERENCE FUNCTION")
print("   - predict_flight_price() created and tested")
print(f"   - Test predictions working correctly")

print("\n2. STREAMLIT APP")
print("   - app.py created with full UI")
print("   - Features: airline, route, stops, date, time selection")
print("   - Displays: predicted price, confidence, price range")
print("   - Includes: tips for cheaper flights")

print("\n3. DEPLOYMENT FILES")
print("   flight_price_predictor/")
for file in os.listdir(deploy_folder):
    file_path = os.path.join(deploy_folder, file)
    size = os.path.getsize(file_path) / 1024
    print(f"   ├── {file} ({size:.1f} KB)")

print("\n4. MODEL DETAILS")
print("   - Algorithm: XGBoost (Tuned)")
print("   - Test R²: 0.9167")
print("   - Test MAE: ₹659")
print("   - Test MAPE: 7.95%")

print("\n5. DEPLOYMENT OPTIONS")
print("   - Local: streamlit run app.py")
print("   - Cloud: Streamlit Cloud (free hosting)")

print("\n" + "="*70)
print("PROJECT COMPLETE!")
print("="*70)
print("""
FULL PIPELINE SUMMARY:
  Phase 1: Data Understanding ✓
  Phase 2: Data Cleaning ✓
  Phase 3: Analysis Questions (8 questions answered) ✓
  Phase 4: ML Preprocessing ✓
  Phase 5: Model Training (XGBoost, R²=0.9167) ✓
  Phase 6: Deployment (Streamlit app) ✓
""")